In [1]:
import xarray as xr
import pandas as pd
import numpy as np
from dask.distributed import Client

In [2]:
# qsub -IVX -l nodes=1:ppn=10,mem=40g,vmem=40g,walltime=01:00:00
# client = Client(n_workers=10, threads_per_worker=1, memory_limit='4GB')
client = Client()
client

Client Scheduler: tcp://127.0.0.1:45141 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 10 Memory: 42.95 GB


In [10]:
client.close()

In [3]:
da = xr.open_mfdataset('/bdd/ERA5/NETCDF/GLOBAL_025/4xdaily/AN_PL/2020/w.2020*.aphe5.GLOBAL_025.nc', parallel=True, chunks={'latitude':100, 'longitude':100}).w
da

,Array,Chunk
Bytes,224.96 GB,183.52 MB
Shape,"(1464, 37, 721, 1440)","(124, 37, 100, 100)"
Count,2892 Tasks,1440 Chunks
Type,float32,numpy.ndarray


- With automatic chunks (on time)
Array	Chunk  
Bytes	224.96 GB	19.05 GB  
Shape	(1464, 37, 721, 1440)	(124, 37, 721, 1440)  
Count	36 Tasks	12 Chunks  
Type	float32	numpy.ndarray  

- With additional chunks over lat/lon
Array	Chunk  
Bytes	224.96 GB	183.52 MB  
Shape	(1464, 37, 721, 1440)	(124, 37, 100, 100)  
Count	2892 Tasks	1440 Chunks  
Type	float32	numpy.ndarray  

In [4]:
# Generate random lat/lon
lats = np.random.uniform(low=-90, high=90, size=(200,))
lons = np.random.uniform(low=0, high=360, size=(200,))

In [5]:
da_latlon = da.sel(latitude=lats, longitude=lons, method='nearest')
da_latlon

/data/mlalande/miniconda3/envs/phd_v3/lib/python3.8/site-packages/xarray/core/indexing.py:1349: PerformanceWarning: Slicing with an out-of-order index is generating 13 times more chunks
  value = value[(slice(None),) * axis + (subkey,)]
/data/mlalande/miniconda3/envs/phd_v3/lib/python3.8/site-packages/xarray/core/indexing.py:1349: PerformanceWarning: Slicing with an out-of-order index is generating 22 times more chunks
  value = value[(slice(None),) * axis + (subkey,)]


,Array,Chunk
Bytes,8.67 GB,220.22 kB
Shape,"(1464, 37, 200, 200)","(124, 37, 3, 4)"
Count,404652 Tasks,383904 Chunks
Type,float32,numpy.ndarray


Array	Chunk  
Bytes	8.67 GB	734.08 MB  
Shape	(1464, 37, 200, 200)	(124, 37, 200, 200)  
Count	60 Tasks	12 Chunks  
Type	float32	numpy.ndarray  

In [8]:
%%time
da_latlon.load()


KeyboardInterrupt

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing


In [ ]:
da_latlon.chunk({'time':-1, 'latitude':50, 'longitude':50, 'levels':10})

In [ ]:
# Interpolate to 3h
da_latlon_interp = da_latlon.interp(time=pd.date_range(da.time[0].values, da.time[-1].values, freq="3H"))

In [6]:
da.time[0]

<xarray.DataArray 'time' ()>
array('2020-01-01T00:00:00.000000000', dtype='datetime64[ns]')
Coordinates:
    time     datetime64[ns] 2020-01-01
Attributes:
    long_name:  time

In [8]:
pd.date_range(da.time[0].values, da.time[-1].values, freq="3H")

DatetimeIndex(['2020-01-01 00:00:00', '2020-01-01 03:00:00',
               '2020-01-01 06:00:00', '2020-01-01 09:00:00',
               '2020-01-01 12:00:00', '2020-01-01 15:00:00',
               '2020-01-01 18:00:00', '2020-01-01 21:00:00',
               '2020-01-02 00:00:00', '2020-01-02 03:00:00',
               ...
               '2020-12-30 15:00:00', '2020-12-30 18:00:00',
               '2020-12-30 21:00:00', '2020-12-31 00:00:00',
               '2020-12-31 03:00:00', '2020-12-31 06:00:00',
               '2020-12-31 09:00:00', '2020-12-31 12:00:00',
               '2020-12-31 15:00:00', '2020-12-31 18:00:00'],
              dtype='datetime64[ns]', length=2927, freq='3H')

In [ ]:
da_latlon.interp(time=)

In [6]:
%%time
da.interp(longitude=20.1, latitude=10.1, time='2020-02-12T18:00:00', method='linear').values

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nann

KeyboardInterrupt: 

distributed.nanny - WARNING - Restarting worker


In [ ]:
%%time
da.interp(longitude=20.1, latitude=10.1, time='2020-02-12T18:00:00', method='linear').values